In [1]:
import nltk

# Download the 'wordnet' resource
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
# Download the 'stopwords' resource
nltk.download('stopwords')

# Download the 'punkt' resource
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
# Install necessary libraries
!pip install pandas nltk gensim

# Import libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load the dataset
url = "https://www.kaggle.com/datasets/nltkdata/movie-review"
# Assuming you have downloaded the dataset and saved it as 'movie_reviews.csv'
df = pd.read_csv('movie_review.csv', delimiter=',')
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,"after watching one of the "" roxbury "" skits on...",neg
64718,9,cv999,14636,23,"bump unsuspecting women , and . . . that's all .",neg


In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))

    # Tokenization and removal of stop words
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]

    return ' '.join(tokens)

In [11]:
df['Text'] = df['text'].apply(preprocess_text)

In [12]:
import numpy as np
tokenized_reviews = [word_tokenize(review) for review in df['Text']]
word2vec_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Vectorization of reviews
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0.

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector

In [13]:
def get_average_word_vectors(reviews, model, vocabulary, num_features):
    review_feature_vectors = [average_word_vectors(review, model, vocabulary, num_features) for review in reviews]
    return np.array(review_feature_vectors)


In [14]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['tag'], test_size=0.2, random_state=42)


In [15]:
# Vectorize reviews
train_vectors = get_average_word_vectors(X_train, word2vec_model, word2vec_model.wv.index_to_key, 100)
test_vectors = get_average_word_vectors(X_test, word2vec_model, word2vec_model.wv.index_to_key, 100)

In [16]:
# Train a classifier (e.g., Logistic Regression)
classifier = LogisticRegression(random_state=42)
classifier.fit(train_vectors, y_train)


LogisticRegression(random_state=42)

In [17]:
# Make predictions
predictions = classifier.predict(test_vectors)


In [18]:

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

In [19]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5334517923362175
Precision: 0.5377420979996671
Recall: 0.5334517923362175
F1 Score: 0.5072685866613987
